<a href="https://colab.research.google.com/github/Angel-Castro-RC/Final_NLP/blob/main/F6_1_NeuralLanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; align-items: flex-start;">
  <div>
      <h1>CS 195: Natural Language Processing</h1>
      <h2>Neural Language Modeling</h2>
      </br>
    <a href="https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_1_NeuralLanguageModeling.ipynb">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
  </div>
  <div style="margin-left: 20px;">
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/dalle_neural_net_viz.png?raw=1" width="500" style="display: block;">
  </div>
</div>


**Cover Illustration:** generated by Dall E using the ChatGPT 4 interface, prompted for a visualization of the network used in the code below. *That's not what I meant.*

## Announcement

AI - English Faculty Candidate: Gabriel Ford

Meeting with students: Thursday at 3:30pm in Howard 309

Scholarly Presentation: Friday at 9:00am in Howard ???

## Reference

SLP: Neural Networks and Neural Language Models, Chapter 7 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/7.pdf



In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.8 MB/s eta 0:00:00


## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [ ]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Review: Text Classification with Keras

Last time, we saw
* how to do text classification when there are more than 2 classes
    - one hot encoded output layer, one node per class, *softmax* output
    - categorical crossentropy loss
* embedding layer
    - pad sequences to all be same length
    - learn vector for each word representing word semantics
    
<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_text_classification.png?raw=1">
</div>

image source: SLP Fig. 7.11, https://web.stanford.edu/~jurafsky/slp3/7.pdf

*pooling* combines/aggregates all of the embeddings in some way

In [ ]:
from datasets import load_dataset
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, GlobalMaxPooling1D, GlobalAveragePooling1D
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
#use one of these instead of Flatten() to try a pooling method
#model.add(GlobalMaxPooling1D())
#model.add(GlobalAveragePooling1D())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
3750/3750 [==============================] - 84s 22ms/step - loss: 0.3262 - accuracy: 0.8795
Epoch 2/10
3750/3750 [==============================] - 80s 21ms/step - loss: 0.1138 - accuracy: 0.9622
Epoch 3/10
2570/3750 [===================>..........] - ETA: 26s - loss: 0.0292 - accuracy: 0.9913

KeyboardInterrupt: ignored

## Neural Language Modeling

**Neural Language Modeling:** predict next word(s) from previous ones - like what we did with Markov models

Like classification, but output is softmax of every possible word in the vocabulary

Often a first step before extending the model to do summarization, translation, dialog, and other NLP tasks

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/neural_language_modeling.png?raw=1">
</div>

image source: SLP Fig. 7.13, https://web.stanford.edu/~jurafsky/slp3/7.pdf

## A Neural Language Model in Keras

Let's start by sampling some data from our news dataset

Then split into a training and testing set

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np
import random

data = load_dataset("ag_news")

data_subset, _ = train_test_split(data["train"]["text"],train_size=500)
train_data, test_data = train_test_split(data_subset,train_size=0.8)

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_subset)
vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary size:",vocabulary_size)

# Convert text to sequences of integers
train_texts = tokenizer.texts_to_sequences(train_data)




Vocabulary size: 5238


## Preparing training examples

We want to take the sequences like

In [ ]:
print( train_texts[0] )

[1917, 1918, 238, 46, 3459, 5, 1207, 127, 34, 34, 3460, 1917, 16, 1918, 238, 46, 3461, 62, 122, 29, 61, 49, 220, 12, 1, 23, 125, 1207, 36, 4, 1, 3462, 3463, 8, 3, 41, 49, 655]


and slide a window across to predict the next word

In [ ]:
print("Use",train_texts[0][0:5],"to predict",train_texts[0][5])
print("Use",train_texts[0][1:6],"to predict",train_texts[0][6])
print("Use",train_texts[0][2:7],"to predict",train_texts[0][7])
print("Use",train_texts[0][3:8],"to predict",train_texts[0][8])
print("etc.")

Use [1917, 1918, 238, 46, 3459] to predict 5
Use [1918, 238, 46, 3459, 5] to predict 1207
Use [238, 46, 3459, 5, 1207] to predict 127
Use [46, 3459, 5, 1207, 127] to predict 34
etc.


## Group Discussion

What data structures (lists, matrixes, etc.) do we need to prepare to make this a classification problem?

*   Input Sequences list of lists or 2D array
*   Labels list or 1D Array
*   Dictionary
*   Embedding Matrix



## Preparing all of the examples

In [ ]:
# Decide the sequence length
sequence_length = 5  # Length of the input sequence before predicting the next word

# Create the sequences
predictor_sequences = []
targets = []
for text in train_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)


print("Number of sequences:",len(predictor_sequences))


print("First train text:",train_texts[0])
print("Example sequence 0:",predictor_sequences[0]," target:",targets[0])
print("Example sequence 1:",predictor_sequences[1]," target:",targets[1])
print("Example sequence 2:",predictor_sequences[2]," target:",targets[2])
print("Example sequence 3:",predictor_sequences[3]," target:",targets[3])
print("Example sequence 4:",predictor_sequences[4]," target:",targets[4])
print("Example sequence 5:",predictor_sequences[5]," target:",targets[5])


Number of sequences: 14050
First train text: [1917, 1918, 238, 46, 3459, 5, 1207, 127, 34, 34, 3460, 1917, 16, 1918, 238, 46, 3461, 62, 122, 29, 61, 49, 220, 12, 1, 23, 125, 1207, 36, 4, 1, 3462, 3463, 8, 3, 41, 49, 655]
Example sequence 0: [1917, 1918, 238, 46, 3459]  target: 5
Example sequence 1: [1918, 238, 46, 3459, 5]  target: 1207
Example sequence 2: [238, 46, 3459, 5, 1207]  target: 127
Example sequence 3: [46, 3459, 5, 1207, 127]  target: 34
Example sequence 4: [3459, 5, 1207, 127, 34]  target: 34
Example sequence 5: [5, 1207, 127, 34, 34]  target: 3460


## Padding

Some of the sequences might be really short - so we'll pad them just in case.

In [ ]:
# Pad sequences to ensure uniform length
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

## The target output

Since we're making this into a classification problem, the output layer needs to have one node for each word in the vocabulary.

Target values need to be transformed into a one-hot encoded vector

In [ ]:

# Convert output to one-hot encoding
target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

print("Predictors words 0:", predictor_sequences_padded[0])
print("target word 0:", targets[0])
print("target word 0 one hot encoded:", target_word_one_hot[0])

Predictors words 0: [1917 1918  238   46 3459]
target word 0: 5
target word 0 one hot encoded: [0. 0. 0. ... 0. 0. 0.]


## Preparing the test set

We have to do all of those same things for the test set.

**Group Exercise:** Turn this into a function so that you can use it to prepare both the training and test sets.

In [ ]:
test_texts = tokenizer.texts_to_sequences(test_data)

# Create the sequences
predictor_sequences_test = []
targets_test = []
for text in test_texts:
    for i in range(sequence_length, len(text)):
        # Take the sequence of tokens as input and the next token as target
        curr_target = text[i]
        curr_predictor_sequence = text[i-sequence_length:i]
        predictor_sequences_test.append(curr_predictor_sequence)
        targets_test.append(curr_target)

# Pad sequences to ensure uniform length
predictor_sequences_padded_test = pad_sequences(predictor_sequences_test, maxlen=sequence_length, padding='pre')

# Convert target to one-hot encoding
target_word_one_hot_test = to_categorical(targets_test, num_classes=vocabulary_size)

In [ ]:
def prepare_sequences(tokenizer, data, sequence_length, vocabulary_size):
    # Convert texts to sequences
    sequences = tokenizer.texts_to_sequences(data)

    # Create predictor sequences and targets
    predictor_sequences = []
    targets = []
    for text in sequences:
        for i in range(sequence_length, len(text)):
            curr_target = text[i]
            curr_predictor_sequence = text[i - sequence_length:i]
            predictor_sequences.append(curr_predictor_sequence)
            targets.append(curr_target)

    # Pad sequences
    predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')

    # Convert targets to one-hot encoding
    target_word_one_hot = to_categorical(targets, num_classes=vocabulary_size)

    return predictor_sequences_padded, target_word_one_hot

# Prepare training set
predictor_sequences_padded, target_word_one_hot = prepare_sequences(tokenizer, train_data, sequence_length, vocabulary_size)

# Prepare test set
predictor_sequences_padded_test, target_word_one_hot_test = prepare_sequences(tokenizer, test_data, sequence_length, vocabulary_size)
print(target_word_one_hot)
print(predictor_sequences_padded)
print (predictor_sequences_padded_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1917 1918  238   46 3459]
 [1918  238   46 3459    5]
 [ 238   46 3459    5 1207]
 ...
 [  20    3  417  623  539]
 [   3  417  623  539   21]
 [ 417  623  539   21  462]]
[[1980  352   30  370  304]
 [ 352   30  370  304 3661]
 [  30  370  304 3661 1980]
 ...
 [   4  471 2159   30 4428]
 [ 471 2159   30 4428   28]
 [2159   30 4428   28    1]]


## Designing the Neural Network

We'll start with a simple network like the one we used for text classification.

In [ ]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=sequence_length))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(vocabulary_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model - you can also pass in the test set
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, verbose=1, validation_data=(predictor_sequences_padded_test, target_word_one_hot_test))

# The model can now be used to predict the next word in a sequence

Epoch 1/10
440/440 [==============================] - 4s 8ms/step - loss: 7.6509 - accuracy: 0.0470 - val_loss: 7.5262 - val_accuracy: 0.0484
Epoch 2/10
440/440 [==============================] - 4s 9ms/step - loss: 6.8347 - accuracy: 0.0504 - val_loss: 7.7602 - val_accuracy: 0.0551
Epoch 3/10
440/440 [==============================] - 3s 8ms/step - loss: 6.4334 - accuracy: 0.0670 - val_loss: 8.0402 - val_accuracy: 0.0726
Epoch 4/10
440/440 [==============================] - 4s 9ms/step - loss: 6.1074 - accuracy: 0.0890 - val_loss: 8.4458 - val_accuracy: 0.0770
Epoch 5/10
440/440 [==============================] - 4s 9ms/step - loss: 5.7834 - accuracy: 0.1081 - val_loss: 8.8913 - val_accuracy: 0.0802
Epoch 6/10
440/440 [==============================] - 3s 8ms/step - loss: 5.4184 - accuracy: 0.1380 - val_loss: 9.4706 - val_accuracy: 0.0813
Epoch 7/10
440/440 [==============================] - 4s 8ms/step - loss: 4.9899 - accuracy: 0.1792 - val_loss: 10.0724 - val_accuracy: 0.0761
Epoch

## Testing the final model on the test set

In [ ]:
loss, accuracy = model.evaluate(predictor_sequences_padded_test, target_word_one_hot_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

108/108 [==============================] - 0s 3ms/step - loss: 13.7035 - accuracy: 0.0653
Test accuracy: 6.53%


## Text Generation

We can use this model to successively generate words based on previous ones - like our Markov sequence on steroids.

Let's see how this works

In [ ]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
print(tokens_list)

tokens_array = np.array(tokens_list)
print(tokens_array)

[[1, 128, 19, 11, 15]]
[[  1 128  19  11  15]]


the model will predict probabilities for each possible word in the output

In [ ]:
predicted_probabilities = model.predict(tokens_array,verbose=0)
print(predicted_probabilities)
print("We get a probability for each of the",len(predicted_probabilities[0]),"words")

[[7.8843922e-20 2.8843084e-07 2.9035034e-03 ... 5.8945300e-11
  5.2877848e-11 8.2093794e-12]]
We get a probability for each of the 5238 words


then we could get the word associated with the highest probability

In [ ]:
predicted_index = np.argmax(predicted_probabilities)
print("word index:",predicted_index)
predicted_word = tokenizer.index_word[predicted_index]
print("word:",predicted_word)

word index: 27
word: was


or you could generate a random word according the these probabilities (like with did with Markov text generation)

### putting it all together

In [ ]:
starter_string = "the government said that it"
tokens_list = tokenizer.texts_to_sequences([starter_string])
tokens = tokens_list[0]

for i in range(50):
    curr_seq = tokens[-sequence_length:]
    curr_array = np.array([curr_seq])
    predicted_probabilities = model.predict(curr_array,verbose=0)
    predicted_index = np.argmax(predicted_probabilities)
    predicted_word = tokenizer.index_word[predicted_index]
    print(predicted_word+" ",end="")
    tokens.append(predicted_index)


would introduce a definitive jury in the 12th season now the surface of a bloody suspension mine in georgia that it will keep him to proceed for the sky oil assets to 2006 what might offer an initial four time in the united states towards the new braves on wednesday 

## Applied Exploration

Pick another dataset and get it working with this code
* you will likely need to prepare the text a little differently - do you need to first break it into sentences?
* describe your dataset and what you did to prepare it

The Reuters dataset is a collection of newswire articles labeled with various topics, used for training a language model.


Perform a neural language modeling experiment
* experiment with something to try to find a well-performing model
    * sliding window size
    * number of hidden nodes in the network
    * learning rate
* describe what you did and write up an interpretation of your results

In [ ]:
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_words = 10000  # Number of words to consider as features
sequence_length = 10  # Sliding window size

# Load Reuters dataset
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=max_words)

# Convert integer sequences back to text
word_index = reuters.get_word_index()
reverse_word_index = {index: word for word, index in word_index.items()}
decoded_newswire = ' '.join([reverse_word_index.get(index - 3, '?') for index in train_data[0]])

# Tokenize the text data
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts([decoded_newswire])  # Use the decoded text for fitting the tokenizer
sequences = tokenizer.texts_to_sequences([decoded_newswire])  # Use the decoded text for tokenization

# Preprocess sequences for sliding window approach
predictor_sequences, targets = [], []

for sequence in sequences:
    for i in range(sequence_length, len(sequence)):
        curr_target = sequence[i]
        curr_predictor_sequence = sequence[i - sequence_length:i]
        predictor_sequences.append(curr_predictor_sequence)
        targets.append(curr_target)

# Padding and one-hot encoding
predictor_sequences_padded = pad_sequences(predictor_sequences, maxlen=sequence_length, padding='pre')
target_word_one_hot = to_categorical(targets, num_classes=max_words)

550378/550378 [==============================] - 0s 0us/step


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

vocabulary_size = 10000  # Same as the 'num_words' parameter used in Tokenizer
embedding_dim = 50
hidden_units = 100

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length))
model.add(LSTM(hidden_units))
model.add(Dense(vocabulary_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(predictor_sequences_padded, target_word_one_hot, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
2/2 [==============================] - 0s 64ms/step - loss: 3.9754 - accuracy: 0.0847 - val_loss: 8.1475 - val_accuracy: 0.0667
Epoch 2/10
2/2 [==============================] - 0s 44ms/step - loss: 3.8128 - accuracy: 0.0847 - val_loss: 8.3200 - val_accuracy: 0.0667
Epoch 3/10
2/2 [==============================] - 0s 37ms/step - loss: 3.7065 - accuracy: 0.0847 - val_loss: 8.5000 - val_accuracy: 0.0667
Epoch 4/10
2/2 [==============================] - 0s 45ms/step - loss: 3.6373 - accuracy: 0.0847 - val_loss: 8.6744 - val_accuracy: 0.0667
Epoch 5/10
2/2 [==============================] - 0s 39ms/step - loss: 3.5895 - accuracy: 0.0847 - val_loss: 8.8386 - val_accuracy: 0.0667
Epoch 6/10
2/2 [==============================] - 0s 41ms/step - loss: 3.5573 - accuracy: 0.0847 - val_loss: 8.9883 - val_accuracy: 0.0667
Epoch 7/10
2/2 [==============================] - 0s 44ms/step - loss: 3.5404 - accuracy: 0.0847 - val_loss: 9.1194 - val_accuracy: 0.0667
Epoch 8/10
2/2 [===========

In [ ]:
import numpy as np

seed_text = "the economy in "
seed_sequence = tokenizer.texts_to_sequences([seed_text])[0]
for _ in range(10):  # Generate 10 words
    padded_sequence = pad_sequences([seed_sequence], maxlen=sequence_length, padding='pre')
    predicted_probabilities = model.predict(padded_sequence, verbose=0)[0]
    predicted_index = np.argmax(predicted_probabilities)
    predicted_word = reverse_word_index.get(predicted_index + 3, "?")  # Adding 3 because of reserved indices
    print(predicted_word, end=" ")
    seed_sequence.append(predicted_index)
    seed_sequence = seed_sequence[2:]  # Shift the sequence by one word

in in in in in in in in in in 

In [ ]:
from keras import optimizers

#an example on changing the learning rate
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=["accuracy"])